In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from scipy.stats import pearsonr
from scipy import stats
import warnings
from scipy.signal import argrelextrema
from cydets.algorithm import detect_cycles
import os 
import openpyxl

warnings.filterwarnings('ignore')

## Load Data

In [37]:
# feature
curr_path = os.getcwd()
filename_list = ['GDP.xlsx', 'IPMAN.xlsx', '10YearTreasury.xlsx', 'AMTMNO.xlsx', 'CPIFABSL.xlsx', 'MedianUsualWeeklyRealEarnings.xlsx',
                'PCE.xlsx', 'PPIACO.xlsx', 'TTLCONS.xlsx', 'VolumeOfTotalRetailTradeSales.xlsx', 'median_new_housing_sales.xlsx', 'unemployment.xlsx']
df_list = []
for i, file in enumerate(filename_list):
    input_path = os.path.join(curr_path,file)
    df = pd.read_excel(
        input_path,
        engine='openpyxl',
        #index_col=0
    )
    #df.set_index(df.columns[0])
    df_list.append(df)


In [38]:
# target variable
input_path = os.path.join(curr_path,'S&P500-EPS.xlsx')
eps_df = pd.read_excel(
    input_path,
    engine='openpyxl',
    header=None,
    skiprows = 1,
    #index_col=0
)
eps_df.columns = ['Date', 'SP500-PE', 'SP500-Price', 'SP500-EPS', 'SP500-Index']
eps_df.dropna(axis=0, how='any', inplace=True)

eps_df = eps_df.iloc[::-1]
eps_df['EPS-GR'] = eps_df['SP500-EPS'].pct_change(1) * 100

In [50]:
# left join features on the target table
t = eps_df
for df in df_list:
    t = pd.merge(t, df, left_on = 'Date', right_index = False, right_on = df.columns[0], how="left")

In [51]:
t

,Date,SP500-PE,SP500-Price,SP500-EPS,SP500-Index,EPS-GR,1969-07-01 00:00:00,1029.956,observation_date_x,IPMAN,...,observation_date_y,PCE,observation_date_x,PPIACO,observation_date_y,TTLCONS,observation_date,SLRTTO01USQ657S,HousingSales,UnemploymentRate
0,1969-08-01,16.04,94.18,5.871571,100.000000,NaN,NaT,NaN,NaT,NaN,...,1969-08-01,609.9,1969-08-01,35.7,NaT,NaN,NaT,NaN,24900,3.5
1,1969-09-01,16.05,94.51,5.888474,100.287869,0.287869,NaT,NaN,NaT,NaN,...,1969-09-01,613.2,1969-09-01,35.8,NaT,NaN,NaT,NaN,26000,3.7
2,1969-10-01,16.32,95.52,5.852941,99.682710,-0.603422,1969-10-01,1038.147,NaT,NaN,...,1969-10-01,618.5,1969-10-01,35.9,NaT,NaN,1969-10-01,1.129686,24400,3.7
3,1969-11-01,16.54,96.21,5.816808,99.067314,-0.617355,NaT,NaN,NaT,NaN,...,1969-11-01,620.5,1969-11-01,36.1,NaT,NaN,NaT,NaN,24900,3.5
4,1969-12-01,15.76,91.11,5.781091,98.459021,-0.614020,NaT,NaN,NaT,NaN,...,1969-12-01,622.8,1969-12-01,36.3,NaT,NaN,NaT,NaN,25100,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2020-11-01,37.16,3548.99,95.505651,1626.577454,-1.385044,NaT,NaN,2020-11-01,96.6509,...,2020-11-01,14467.3,2020-11-01,198.3,2020-11-01,1487171.0,NaT,NaN,350800,6.7
616,2020-12-01,39.26,3695.31,94.124045,1603.047015,-1.446623,NaT,NaN,2020-12-01,97.3233,...,2020-12-01,14389.5,2020-12-01,200.5,2020-12-01,1504188.0,NaT,NaN,365300,6.7
617,2021-01-01,40.30,3793.75,94.137717,1603.279871,0.014526,2021-01-01,22038.226,2021-01-01,98.7911,...,2021-01-01,14857.9,2021-01-01,204.8,2021-01-01,1549793.0,NaT,NaN,373200,6.3
618,2021-02-01,41.26,3883.43,94.120940,1602.994143,-0.017821,NaT,NaN,2021-02-01,94.9946,...,2021-02-01,14699.6,2021-02-01,210.6,2021-02-01,1533252.0,NaT,NaN,362000,6.2


In [52]:
df_list[9]

,observation_date,SLRTTO01USQ657S
0,1960-01-01,-5.489193
1,1960-04-01,0.999929
2,1960-07-01,-1.454278
3,1960-10-01,-0.907884
4,1961-01-01,-0.977896
...,...,...
229,2017-04-01,0.644545
230,2017-07-01,0.639493
231,2017-10-01,1.674883
232,2018-01-01,-0.551374
